In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `LLMEngineering` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authen

# ContactDoctor Bio Medical LLM

In [2]:
!pip install -q transformers accelerate bitsandbytes gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.2 MB/s eta 0:00:00


In [3]:
!pip install peft

### Load the data from the file to train the model

In [4]:
from datasets import Dataset

data = [
    {"input": "i have a data\nTotal Steps: 75000\nAverage Zone Minutes: 90\nAverage Heart Rate: 72 bpm\nfood consumption of the week: varied, but often feel bloated\nSleep activity for the week: 7 hours avg, generally good\nPhysical activities performed for the week: walking, some yoga\n\nPlease provide:\n1. Food Recommendation\n2. Physical Exercise\n3. Social Wellbeing Recommendation\n4. Overall Suggestion","output": "1. Food Recommendation:\n- Your **varied** diet is a good foundation, but the symptom of often feeling **bloated** suggests a need to investigate specific food sensitivities or eating habits.\n- **Keep a Food Diary:** For a week or two, track what you eat and when you experience bloating. This can help identify trigger foods.\n- **Slow Down Eating:** Eat slowly and mindfully, chewing your food thoroughly. Rushing can lead to swallowing air and bloating.\n- **Limit Gas-Producing Foods (initially):** Temporarily reduce intake of common culprits like beans, lentils, broccoli, cabbage, and carbonated drinks. Reintroduce them slowly to see what you tolerate.\n- **Consider Probiotics:** Incorporate fermented foods (yogurt, kefir, sauerkraut) or a probiotic supplement to support gut health.\n- **Stay Hydrated:** Adequate water intake aids digestion and can prevent constipation, a cause of bloating.\n\n2. Physical Exercise:\n- Your **Total Steps: 75000** and **Average Zone Minutes: 90** indicate a good level of moderate activity (walking, some yoga). Your **Average Heart Rate of 72 bpm** is healthy.\n- **Increase Intensity Gradually:** While consistent, consider adding short bursts of more intense activity within your walks to boost your cardiovascular fitness further and increase your **Zone Minutes**.\n- **Incorporate Strength Training:** Adding 2-3 sessions of strength training per week will build muscle, improve metabolism, and support bone health, complementing your current activities.\n- **Explore Different Yoga Styles:** If you're comfortable, try different yoga styles (e.g., Vinyasa, Power Yoga) to challenge your body in new ways and potentially aid digestion through specific poses.\n\n3. Social Wellbeing Recommendation:\n- Your data doesn't directly indicate social wellbeing issues, and your **7 hours avg, generally good** sleep is a positive sign for overall energy levels to engage socially.\n- **Leverage Your Activities:** Consider joining a yoga studio or a walking group. This is a great way to meet like-minded people and integrate social interaction with your physical activities.\n- **Plan Regular Social Connections:** Even if you feel generally good, proactively scheduling time with friends and family can enhance your sense of belonging and reduce stress.\n- **Practice Mindful Socializing:** Be present during interactions and avoid distractions, which can deepen connections.\n\n4. Overall Suggestion:\n- Your primary focus should be on addressing the **bloating and digestive discomfort**, as this can significantly impact your daily comfort and nutrient absorption. Your exercise and sleep habits are generally positive, providing a good foundation.\n- **Digestive Health First:** Work on identifying and managing the causes of your bloating through dietary adjustments and possibly consulting a healthcare professional or registered dietitian.\n- **Mindful Eating Practices:** Beyond what you eat, how you eat (slowly, without distractions) is crucial for digestion.\n- **Holistic Wellness:** Continue your balanced approach to physical activity. The combination of walking and yoga is beneficial for both physical and mental health, which can indirectly support digestive health.\n- **Listen to Your Body's Cues:** Pay attention not just to major symptoms but also to subtle signals your body gives you regarding food and activity."}
]

dataset = Dataset.from_list(data)


In [5]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

# === Step 1: Model & Tokenizer Setup ===
model_id = "AnjaliNV/WellBeing_LLM"  # or base model like "mistralai/Mistral-7B-Instruct-v0.1"

print("🔄 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

def preprocess(example):
    prompt = example["input"]
    response = example["output"]
    text = f"### Question: {prompt}\n### Answer: {response}"
    tokens = tokenizer(text, truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(preprocess)
print("🔄 Tokenizing dataset...")
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

# tokenized_dataset = formatted_dataset.map(tokenize, batched=True)

# === Step 2: Load Quantized Base Model ===
print("🔄 Loading base model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)

# === Step 3: Inject LoRA ===
print("🧠 Adding LoRA adapters...")
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# === Step 4: Set Up Trainer ===
print("⚙️ Preparing trainer...")
training_args = TrainingArguments(
    output_dir="./WellBeing_LLM_finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    learning_rate=3e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to=[],
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# === Step 5: Train ===
print("🚀 Starting training...")
trainer.train()

# === Step 6: Save Final Model + Tokenizer ===
print("💾 Saving the fine-tuned model...")
trainer.save_model("./WellBeing_LLM_finetuned")
tokenizer.save_pretrained("./WellBeing_LLM_finetuned")


🔄 Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/359 [00:00<?, ?B/s]

...

/tmp/ipython-input-5-1475796900.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.0848
⚙️ Preparing trainer...
🚀 Starting training...


Step,Training Loss


💾 Saving the fine-tuned model...


('./WellBeing_LLM_finetuned/tokenizer_config.json',
 './WellBeing_LLM_finetuned/special_tokens_map.json',
 './WellBeing_LLM_finetuned/chat_template.jinja',
 './WellBeing_LLM_finetuned/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_id = "AnjaliNV/WellBeing_LLM"
adapter_path = "./WellBeing_LLM_finetuned"

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, adapter_path)
model.eval()


In [8]:
import gradio as gr
import torch

def chat_only_interface(message, history):
    prompt_template = f"""
You are a wellbeing adviser. Respond to the user's condition using the following format:

1. Food Recommendation:
2. Physical Exercise:
3. Social Wellbeing Recommendation:
4. Overall Suggestion:

Example:
User Input: "Total Steps: 75000, Average Zone Minutes: 90, Average Heart Rate: 72 bpm, food consumption of the week: varied, but often feel bloated, Sleep activity for the week: 7 hours avg, generally good, Physical activities performed for the week: walking, some yoga"

1. Food Recommendation:
- Keep a food diary to track bloating triggers.
- Eat slowly and avoid gas-producing foods initially.
- Include probiotics like yogurt or kefir.
- Stay well-hydrated.

2. Physical Exercise:
- You are doing well. Consider adding intensity with interval walks.
- Try Vinyasa yoga for digestive benefits.
- Add strength training twice a week.

3. Social Wellbeing Recommendation:
- Join a yoga or walking group for social interaction.
- Regularly connect with friends/family.

4. Overall Suggestion:
- Focus on resolving bloating.
- Maintain current sleep and activity patterns.
- Be mindful and listen to your body.

Now, based on the following:

User Input: "{message}"

Respond in the same format.
"""

    inputs = tokenizer(prompt_template, return_tensors="pt").to(model.device)

    outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.2,
)


    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = response[len(prompt_template):].strip()

    with open("chat_history.txt", "a", encoding="utf-8") as f:
        f.write(f"User: {message}\n")
        f.write(f"Model: {generated_text}\n\n")

    return generated_text

gr.ChatInterface(
    fn=chat_only_interface,
    title="🧠 Well Being LLM",
    description="Receive holistic health advice from your personalized wellbeing model.",
).queue().launch(debug=True, share=True)


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8a4186f5d40a103eee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8a4186f5d40a103eee.gradio.live


In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("WellBeing_LLM")
tokenizer = AutoTokenizer.from_pretrained("WellBeing_LLM")

# Push to HF Hub
model.push_to_hub("WellBeing_LLM")        # this creates a repo under your username
tokenizer.push_to_hub("WellBeing_LLM")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at WellBeing_LLM and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AnjaliNV/WellBeing_LLM/commit/d622c5abe3666e20f66d784d2674467753d77d1c', commit_message='Upload tokenizer', commit_description='', oid='d622c5abe3666e20f66d784d2674467753d77d1c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AnjaliNV/WellBeing_LLM', endpoint='https://huggingface.co', repo_type='model', repo_id='AnjaliNV/WellBeing_LLM'), pr_revision=None, pr_num=None)